In [1]:
using Pkg; Pkg.activate("v1.5")

 Activating environment at `~/.julia/dev/SymArrays/examples/v1.5/Project.toml`


In [2]:
using Printf
using BenchmarkTools
using SymArrays
using SymArrays: SymIndexIter
println("Julia version $VERSION")

Julia version 1.5.1


In [3]:
sumfirstinds(iter) = (s=0; for I in iter s+=first(Tuple(I)) end; s)
sumvals(iter) = reduce(+, iter);

In [4]:
ttt(iter) = (s=0; for ii in 1:length(iter) s+=first(SymArrays.ind2sub_symgrp(iter,ii)) end; s);

In [5]:
@btime ttt(SI)          setup=(SI=SymIndexIter(4,50))

  14.329 ms (0 allocations: 0 bytes)


3162510

In [6]:
@btime sumfirstinds(SI) setup=(SI=SymIndexIter(4,50))

  54.309 μs (0 allocations: 0 bytes)


3162510

In [7]:
for n in (10, 50, 200)
    n != 10 && println()
    println("n = $n:")
    for N in 1:4, M in (N,-N)
        iter = SymIndexIter(M,n)
        # check that number of elements is correct
        @assert count(x->true,iter) == length(iter)

        @printf("  M = %2d:", M)
        @btime sumfirstinds($iter)
    end
end

n = 10:
  M =  1:  1.811 ns (0 allocations: 0 bytes)
  M = -1:  1.813 ns (0 allocations: 0 bytes)
  M =  2:  29.670 ns (0 allocations: 0 bytes)
  M = -2:  21.816 ns (0 allocations: 0 bytes)
  M =  3:  138.932 ns (0 allocations: 0 bytes)
  M = -3:  112.557 ns (0 allocations: 0 bytes)
  M =  4:  636.545 ns (0 allocations: 0 bytes)
  M = -4:  288.615 ns (0 allocations: 0 bytes)

n = 50:
  M =  1:  2.164 ns (0 allocations: 0 bytes)
  M = -1:  2.164 ns (0 allocations: 0 bytes)
  M =  2:  118.159 ns (0 allocations: 0 bytes)
  M = -2:  128.920 ns (0 allocations: 0 bytes)
  M =  3:  3.165 μs (0 allocations: 0 bytes)
  M = -3:  2.909 μs (0 allocations: 0 bytes)
  M =  4:  59.724 μs (0 allocations: 0 bytes)
  M = -4:  48.967 μs (0 allocations: 0 bytes)

n = 200:
  M =  1:  1.879 ns (0 allocations: 0 bytes)
  M = -1:  1.811 ns (0 allocations: 0 bytes)
  M =  2:  491.714 ns (0 allocations: 0 bytes)
  M = -2:  456.335 ns (0 allocations: 0 bytes)
  M =  3:  44.040 μs (0 allocations: 0 bytes)
  M = -

In [8]:
for S in (SymArray{(3,),Float64}(50,50,50),
          SymArray{(-3,),Float64}(50,50,50),
          SymArray{(3,-2,2),Float64}(2,2,2,3,3,4,4),
          SymArray{(3,2,2),Float64}(2,2,2,3,3,4,4),
          SymArray{(-3,-4),Float64}(8,8,8,5,5,5,5),
          SymArray{(3,-2,2),Float64}(2,2,2,30,30,4,4),
          SymArray{(3,2,2),Float64}(2,2,2,30,30,4,4),
          SymArray{(-3,-4),Float64}(20,20,20,5,5,5,5))
    @printf("typeof(S): %-54s, size(S): %s, length(S): %d, full length(S): %d\n", typeof(S), size(S), length(S), prod(size(S)))
    fill!(S,1.)
    @btime sumfirstinds($(eachindex(S)))
    @btime sumvals($S)
end

typeof(S): SymArray{(3,),Float64,3,1,Array{Float64,1}}           , size(S): (50, 50, 50), length(S): 22100, full length(S): 125000
  3.089 μs (0 allocations: 0 bytes)
  151.094 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(-3,),Float64,3,1,Array{Float64,1}}          , size(S): (50, 50, 50), length(S): 19600, full length(S): 125000
  3.077 μs (0 allocations: 0 bytes)
  156.750 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(3, -2, 2),Float64,7,3,Array{Float64,3}}     , size(S): (2, 2, 2, 3, 3, 4, 4), length(S): 120, full length(S): 1152
  450.576 ns (0 allocations: 0 bytes)
  2.041 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(3, 2, 2),Float64,7,3,Array{Float64,3}}      , size(S): (2, 2, 2, 3, 3, 4, 4), length(S): 240, full length(S): 1152
  981.769 ns (0 allocations: 0 bytes)
  3.235 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(-3, -4),Float64,7,2,Array{Float64,2}}       , size(S): (8, 8, 8, 5, 5, 5, 5), length(S): 280, full length(S): 320000
  329.154 ns (0 allocations: 0 b